In [1]:
%%bash
mkdir src
mkdir report
mkdir build
rm -rf sample_data/

# Naive Approach

## Without share memory

In [25]:
%%bash 

hipcc -O3 -std=c++17 src/naive-main.cpp -o build/naive-main --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/naive-main-${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/naive-main -m $M -n $N -k $K >> $output 2>&1
done

src/naive-main.cpp:61:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
   61 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/naive-main.cpp:154:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  154 |   hipMalloc((int **)&d_a, M * K * sizeof(int));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/naive-main.cpp:155:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  155 |   hipMalloc((int **)&d_b, K * N * sizeof(int));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/naive-main.cpp:156:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  156 |   hipMalloc((int **)&d_c, M * N* sizeof(int));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/naive-main.cpp:162:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-r

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192
Error while terminating subprocess (pid=1557213): 


TypeError: %d format: a real number is required, not NoneType

## With share memory

In [19]:
%%bash 

hipcc -O3 -std=c++17 src/naive-share.cpp -o build/naive-share --offload-arch=gfx90a
srun --time=1:00 ./build/naive-share -m 10 -n 10 -k 10 > report/naive-share.log

src/naive-share.cpp:17:32: warning: variable length arrays in C++ are a Clang extension [-Wvla-cxx-extension]
   17 |     extern __shared__ T As[16][K];
      |                                ^
src/naive-share.cpp:17:32: note: function parameter 'K' with unknown value cannot be used in a constant expression
src/naive-share.cpp:13:79: note: declared here
   13 | __global__ void matmul_kernel(const T *a, const T *b, T *c, int M, int N, int K) {
      |                                                                               ^
src/naive-share.cpp:17:12: error: __shared__ variable 'As' cannot be 'extern'
   17 |     extern __shared__ T As[16][K];
      |            ^
/opt/rocm-6.3.3/lib/llvm/lib/clang/18/include/__clang_hip_runtime_wrapper.h:24:35: note: expanded from macro '__shared__'
   24 | #define __shared__ __attribute__((shared))
      |                                   ^
src/naive-share.cpp:17:25: error: variably modified type declaration cannot have 'extern' linkage
   17 | 

In [2]:
%%bash 

hipcc -O3 -std=c++17 src/tiled-main.cpp -o build/tiled-main --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/tiled-main-${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/tiled-main -m $M -n $N -k $K >> $output 2>&1
done

src/tiled-main.cpp:100:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  100 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/tiled-main.cpp:187:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  187 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-main.cpp:188:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  188 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-main.cpp:189:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  189 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-main.cpp:195:3: warning: ignoring return value of function declared with 'nodis

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192


CalledProcessError: Command 'b'\nhipcc -O3 -std=c++17 src/tiled-main.cpp -o build/tiled-main --offload-arch=gfx90a\nPARAMS=(\n    "1024 1024 1024"\n    "1024 1024 128"\n    "512 2048 4096"\n    "8192 8192 8192"\n)\n\n# Create output directory if it doesn\'t exist.\nmkdir -p report\n\n# Loop over each parameter set.\nfor params in "${PARAMS[@]}"; do\n    # Parse M, N, K.\n    set -- $params\n    M=$1\n    N=$2\n    K=$3\n\n    output="report/tiled-main-${M}_${N}_${K}.txt"\n    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output\n    # Run the executable with srun (adjust --time if needed)\n    srun --time=1:00 ./build/tiled-main -m $M -n $N -k $K >> $output 2>&1\ndone\n'' returned non-zero exit status 143.

In [3]:
%%bash 

hipcc -O3 -std=c++17 src/tiled-ilp-1D-main.cpp -o build/tiled-ilp-1D-main --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/tiled-ilp-1D-main-${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/tiled-ilp-1D-main -m $M -n $N -k $K >> $output 2>&1
done

src/tiled-ilp-1D-main.cpp:108:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  108 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-1D-main.cpp:189:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  189 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-1D-main.cpp:190:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  190 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-1D-main.cpp:191:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  191 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-1D-main.cpp:197:3: warning: ignoring return val

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192


In [5]:
%%bash 

hipcc -O3 -std=c++17 src/tiled-ilp-2D-main.cpp -o build/tiled-ilp-2D-main --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/tiled-ilp-2D-main-${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/tiled-ilp-2D-main -m $M -n $N -k $K >> $output 2>&1
done

src/tiled-ilp-2D-main.cpp:125:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  125 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-2D-main.cpp:212:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  212 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-2D-main.cpp:213:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  213 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-2D-main.cpp:214:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  214 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/tiled-ilp-2D-main.cpp:220:3: warning: ignoring return val

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Error while terminating subprocess (pid=1696467): 


TypeError: %d format: a real number is required, not NoneType

Running configuration: M=8192, N=8192, K=8192


In [9]:
%%bash 

hipcc -O3 -std=c++17 src/vectorize.cpp -o build/vectorize --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/vectorize_${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/vectorize -m $M -n $N -k $K >> $output 2>&1
done

src/vectorize.cpp:101:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  101 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/vectorize.cpp:186:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  186 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/vectorize.cpp:187:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  187 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/vectorize.cpp:188:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  188 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/vectorize.cpp:194:3: warning: ignoring return value of function declared with 'nodiscard'

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192
Error while terminating subprocess (pid=1824945): 


TypeError: %d format: a real number is required, not NoneType

In [7]:
%%bash 

hipcc -O3 -std=c++17 src/warp-tiling-main.cpp -o build/warp-tiling-main --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/warp-tiling-main_${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/warp-tiling-main -m $M -n $N -k $K >> $output 2>&1
done

src/warp-tiling-main.cpp:199:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  199 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/warp-tiling-main.cpp:286:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  286 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/warp-tiling-main.cpp:287:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  287 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/warp-tiling-main.cpp:288:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  288 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/warp-tiling-main.cpp:294:3: warning: ignoring return value of

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192


CalledProcessError: Command 'b'\nhipcc -O3 -std=c++17 src/warp-tiling-main.cpp -o build/warp-tiling-main --offload-arch=gfx90a\nPARAMS=(\n    "1024 1024 1024"\n    "1024 1024 128"\n    "512 2048 4096"\n    "8192 8192 8192"\n)\n\n# Create output directory if it doesn\'t exist.\nmkdir -p report\n\n# Loop over each parameter set.\nfor params in "${PARAMS[@]}"; do\n    # Parse M, N, K.\n    set -- $params\n    M=$1\n    N=$2\n    K=$3\n\n    output="report/warp-tiling-main_${M}_${N}_${K}.txt"\n    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output\n    # Run the executable with srun (adjust --time if needed)\n    srun --time=1:00 ./build/warp-tiling-main -m $M -n $N -k $K >> $output 2>&1\ndone\n'' returned non-zero exit status 143.

In [11]:
%%bash 

hipcc -O3 -std=c++17 src/best-gemm.cpp -o build/best-gemm --offload-arch=gfx90a
PARAMS=(
    "1024 1024 1024"
    "1024 1024 128"
    "512 2048 4096"
    "8192 8192 8192"
)

# Create output directory if it doesn't exist.
mkdir -p report

# Loop over each parameter set.
for params in "${PARAMS[@]}"; do
    # Parse M, N, K.
    set -- $params
    M=$1
    N=$2
    K=$3

    output="report/best-gemm_${M}_${N}_${K}.txt"
    echo "Running configuration: M=$M, N=$N, K=$K" | tee $output
    # Run the executable with srun (adjust --time if needed)
    srun --time=1:00 ./build/best-gemm -m $M -n $N -k $K >> $output 2>&1
done

src/best-gemm.cpp:112:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  112 |   hipDeviceSynchronize();
      |   ^~~~~~~~~~~~~~~~~~~~
src/best-gemm.cpp:197:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  197 |   hipMalloc((float **)&d_a, M * K * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/best-gemm.cpp:198:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  198 |   hipMalloc((float **)&d_b, K * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/best-gemm.cpp:199:3: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
  199 |   hipMalloc((float **)&d_c, M * N * sizeof(float));
      |   ^~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/best-gemm.cpp:205:3: warning: ignoring return value of function declared with 'nodiscard'

Running configuration: M=1024, N=1024, K=1024
Running configuration: M=1024, N=1024, K=128
Running configuration: M=512, N=2048, K=4096
Running configuration: M=8192, N=8192, K=8192
